In [ ]:
import requests
import re
import urllib 
from bs4 import BeautifulSoup
import string
import pandas as pd
import numpy as np
import datetime
import _datetime

In [ ]:
current_datetime1 = datetime.datetime.now()
temporary_links = []
primary_links = []
product_links = []
product_links1 = []

url='https://www.car-tool.ru/catalog'
r = requests.get(url)
html_content = r.text
soup = BeautifulSoup(html_content,'html.parser')
k=0
mm=0
for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_='menu_left'):
    temporary_links.append(link.get('href'))
print (len(temporary_links))
for temporary_link in np.unique(temporary_links):
    url='https://www.car-tool.ru'+ temporary_link
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')
    if soup.findAll('div', class_="modern-page-navigation")==[]:
        last_pages=1
    else:
        a=str(soup.findAll('div', class_="modern-page-navigation")[0]).split()[-9]
        last_pages=a[a.find('>')+1:a.find('<')]
    for pages in range(1,int(last_pages)+1):
        url='https://car-tool.ru'+temporary_link+'?PAGEN_2='+str(pages)
        r = requests.get(url)
        html_content = r.text
        soup = BeautifulSoup(html_content,'html.parser')
        for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_='img-main'):
            product_links1.append(link.get('href'))
    k=k+1        
    print(k,url)
print (len(product_links1))


In [ ]:
titles = []
prices = []
marks=[]
articles=[]
valutes=[]
categories=[]

In [ ]:
for product_link in product_links1:
    
    
    url = 'https://www.car-tool.ru' + product_link
    try:
        r = requests.get(url)
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
        print("pobeda")
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')

    title = soup.findAll('h1', attrs={'id': 'pagetitle'})[0].text
    titles.append(title)


    article = soup.findAll('span', class_='ffen')[0].text
    articles.append(article)

    price=soup.findAll('span',class_="right")[0].text
    prices.append(price)

    if soup.findAll('a', attrs={'title': not None, 'href': not None},class_=None)==[]:
        category="Главная"
    else:
        category=soup.findAll('a', attrs={'title': not None, 'href': not None},class_=None)[-1].text
    categories.append(category)
    mm=mm+1        
    print(mm)

product_links =[ 'https://www.car-tool.ru%s'%x for x in product_links1 ]

table = pd.DataFrame(list(zip(titles, articles,prices, categories,product_links)), columns =['Наименование товара', 'Артикул','Цена, руб.','Категория','Ссылка на товар']) 

table.to_excel('car-tools 30.04.xlsx')

current_datetime2 = datetime.datetime.now()
print (current_datetime2 - current_datetime1)